In [1]:
import polars as pl
import pyodbc
from datetime import date

In [12]:
cierre = "202506"
cierre_l = "Jun25"
cierre_d = date(2025,6,30)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/fotos/"
wd_data_validations = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/validations/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_db_dwh_nr = wd_data_processed_fotos + f'parquet/DB_DWH_NR_{cierre}.parquet'
fl_db_dwh_r = wd_data_processed_fotos + f'parquet/DB_DWH_R_{cierre}.parquet'
fl_prem = wd_data_raw + f'Prem_{cierre}.mdb'

fl_recuperadas_detalle = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'
fl_pagadas_detalle = wd_data_processed_dwh + f'parquet/Pagadas_Global_VF_{cierre}.parquet'

# Outputs
fl_base_agrup_final = wd_data_processed_dwh + f'parquet/BASE_AGRUP_FINAL_{cierre}.parquet'
fl_valida_saldo = wd_data_validations + f'VALIDA_SALDO.csv'
fl_bd_dwh_plazorem = wd_data_processed_dwh + f'parquet/BD_DWH_PlazoRem_{cierre}.parquet'
fl_bd_dwh_plazorem_xl = wd_data_processed_dwh + f'parquet/BD_DWH_PlazoRem_{cierre}.xlsx'
fl_valida_saldo_prem = wd_data_validations + f'Valida_Saldo_Prem.csv'




### Macro Saldos

In [27]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

estados = pl.read_database(query='SELECT * FROM ESTADOS', connection=conn, schema_overrides={'Estado_ID': pl.Int32})
sector = pl.read_database(query='SELECT * FROM SECTOR', connection=conn, schema_overrides={'Sector_ID': pl.Int32})
estrato = pl.read_database(query='SELECT * FROM ESTRATO', connection=conn, schema_overrides={'Estrato_Id': pl.Int32})
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

In [28]:
db_dwh_nr = pl.read_parquet(fl_db_dwh_nr)
db_dwh_r = pl.read_parquet(fl_db_dwh_r)

In [29]:
def garantias_c_saldo(df):
    """ Filtra garantías con saldo positivo.  """
    result = df.filter(pl.col("Saldo_Contingente_Mn") > 0)
    return result

db_dwh_nr = garantias_c_saldo(db_dwh_nr)
db_dwh_r = garantias_c_saldo(db_dwh_r)

In [30]:
def cruza_catalogos(df):
    """Incorpora catálogos. """
    # sfc_aux = sfc.select(
    #     pl.col("Intermediario_Id"),
    #     pl.col("CLAVE_CREDITO"),
    #     pl.col("FONDOS_CONTRAGARANTIA").alias("CSF")
    # )

    result = (df.join(estados, left_on="ESTADO_ID", right_on="Estado_ID", how="left")
              .join(estrato, left_on="ESTRATO_ID", right_on="Estrato_Id", how="left")
              .join(sector, left_on="SECTOR_ID", right_on="Sector_ID", how="left")
              .join(tipo_garantia.select(['Tipo_garantia_ID', 'Tipo_garantia']), left_on='TIPO_GARANTIA_ID', right_on='Tipo_garantia_ID', how='left')
              .join(tipo_credito.select(['Tipo_Credito_ID', 'Tipo_Credito']), left_on='TIPO_CREDITO_ID', right_on="Tipo_Credito_ID", how='left')
              #.join(sfc_aux, left_on=['INTERMEDIARIO_ID', 'NUMERO_CREDITO'], right_on=['Intermediario_Id', 'CLAVE_CREDITO'], how='left')
              )
    
    result = result.with_columns(pl.col("CSF").fill_null("CF"))
    return result

db_dwh_r = cruza_catalogos(db_dwh_r)
db_dwh_nr = cruza_catalogos(db_dwh_nr)

In [31]:
def agrupa_bd_con_saldo(df):
    result = df.group_by([
        "BUCKET", 
        "Producto", 
        "TIPO_CREDITO_ID", 
        "MM_UDIS", 
        "TPRO_CLAVE", 
        "NR_R", 
        "CSG", 
        "INTERMEDIARIO_ID", 
        "Razón Social (Intermediario)", 
        "Tipo_Persona", 
        "Porcentaje de Comisión Garantia", 
        "Porcentaje Garantizado", 
        "AGRUPAMIENTO", 
        "Estado", 
        "Sector", 
        "Estrato", 
        "Tipo_garantia", 
        "Tipo_Credito", 
        "PROGRAMA_ID",  
        "CSF"
    ]).agg(
        pl.col("Saldo_Contingente_Mn").sum().alias("SALDO_MN"),
        pl.col("NUMERO_CREDITO").count().alias("TOT_CREDITOS")
    )
    return result

db_dwh_r_vf = agrupa_bd_con_saldo(db_dwh_r)
db_dwh_nr_vf = agrupa_bd_con_saldo(db_dwh_nr)

In [32]:
base_agrup_final = pl.concat([db_dwh_r_vf, db_dwh_nr_vf], rechunk=True)

In [33]:
# VALIDA_SALDO
valida_saldo = base_agrup_final.group_by(
    "Producto"
).agg(
    pl.col("SALDO_MN").sum().alias("S_SALDO(MDP)")
)

In [34]:
base_agrup_final.write_parquet(fl_base_agrup_final)
valida_saldo.write_csv(fl_valida_saldo, float_precision=10, float_scientific=False)

#### Prem

In [4]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_prem
conn = pyodbc.connect(conn_str)
prem = pl.read_database(query='SELECT * FROM DATOS', connection=conn)
conn.close()

In [5]:
bd_dwh_plazorem = (
    prem.with_columns(
        pl.col('BUCKET'),
        pl.col('DESC_INDICADOR').alias('Producto'),
        pl.col('FECHA_APERTURA').alias('Fecha de Apertura'),
        pl.col('FECHA_CONSULTA'),
        pl.col('FVTO_RIESGOSD'),
        pl.col('INDICADOR_ID').alias('Producto ID'),
        pl.col('INTERMEDIARIO_ID'),
        pl.col('ORDEN_FREGISTRO'),
        pl.col('PROGRAMA_ID'),
        pl.col('RAZON_SOCIAL').alias('Razón Social (Intermediario)'),
        pl.col('MONTO_CREDITO_MN (SUMA)').alias('SUM Monto _Credito_Mn'),
        pl.col('NUMERO_CREDITOS (SUMA)').alias('COUNT Numero_Credito'),
        pl.col('SALDO_CONTINGENTE_MN (SUMA)').alias('SUM Saldo_Contingente_Mn'),
    )
)

In [11]:
valida_saldo_prem = (
    bd_dwh_plazorem.group_by('Producto')
    .agg(pl.col('SUM Saldo_Contingente_Mn').sum().alias("S_Saldo"))
)


In [ ]:
bd_dwh_plazorem.write_parquet(fl_bd_dwh_plazorem)
bd_dwh_plazorem.write_excel(fl_bd_dwh_plazorem_xl)
valida_saldo_prem.write_csv(fl_valida_saldo_prem, float_precision=10, float_scientific=False)